**Please install these two packages before running the code**

In [ ]:
pip install overpy

In [ ]:
pip install pandas

**Code of the application eaToday**

In [ ]:
import overpy  # Overpy: To interact with OpenStreetMap (OSM) Overpass API.
import pandas as pd  # Pandas: For data manipulation and storage.
from datetime import datetime, timedelta  # datetime and timedelta: To work with dates and times.
from typing import List, Dict, Tuple  # Typing: To provide type hints for variables and functions.
from math import radians, sin, cos, sqrt, atan2  # Math functions: To calculate distances between two coordinates.
import random  # Random: To generate random numbers, sample lists, etc.

print("Initializing Overpass API Query...")

def fetch_osm_data(query: str):
    """
    Fetch OSM data using the Overpass API.
    :param query: The Overpass query string.
    :return: An Overpy.Result object containing the queried data.
    """
    try:
        api = overpy.Overpass()  # Initialize the Overpass API
        result = api.query(query)  # Execute the query
        return result  # Return the result object
    except overpy.exception.OverpassException as e:
        print(f"Error fetching OSM data: {e}")
        return None

# Query for restaurants and supermarkets in Copenhagen
query = """
[out:json];
(
  node["amenity"="restaurant"](55.6, 12.4, 55.8, 12.8);
  node["shop"="supermarket"](55.6, 12.4, 55.8, 12.8);
  way["amenity"="restaurant"](55.6, 12.4, 55.8, 12.8);
  way["shop"="supermarket"](55.6, 12.4, 55.8, 12.8);
  relation["amenity"="restaurant"](55.6, 12.4, 55.8, 12.8);
  relation["shop"="supermarket"](55.6, 12.4, 55.8, 12.8);
);
out body;
>;
"""

# Fetch OSM data based on the query.
result = fetch_osm_data(query)

# Initialize an empty list to store extracted shop data.
shop_data = []

if result:
    print("Processing OSM data...")
    try:
        # Process nodes (e.g., small points of interest).
        for node in result.nodes:
            name = node.tags.get("name", "N/A")
            latitude = node.lat
            longitude = node.lon
            shop_data.append({"name": name, "latitude": latitude, "longitude": longitude})

        # Process ways (e.g., buildings or larger mapped areas).
        for way in result.ways:
            name = way.tags.get("name", "N/A")
            way_nodes = way.get_nodes(resolve_missing=True)
            if way_nodes:
                first_node = way_nodes[0]
                latitude = first_node.lat
                longitude = first_node.lon
            else:
                latitude = longitude = "N/A"
            shop_data.append({"name": name, "latitude": latitude, "longitude": longitude})
        print("OSM data processed successfully.")
    except Exception as e:
        print(f"Error processing OSM data: {e}")

# Create a DataFrame for food allergies
print("Creating allergy data...")
try:
    allergies_data = {
        "Allergen": [
            "Peanuts", "Tree Nuts", "Milk", "Eggs", "Fish", "Shellfish", 
            "Wheat", "Soy", "Sesame", "Mustard", "Celery", "Sulphites", "Lupin"
        ],
        "Description": [
            "A common legume that can cause severe allergic reactions.",
            "Nuts that grow on trees, such as almonds, walnuts, and cashews.",
            "Dairy products, including cheese, butter, and yogurt.",
            "Found in many baked goods, mayonnaise, and egg-based dishes.",
            "Fish such as salmon, tuna, and cod can trigger allergies.",
            "Includes shrimp, crab, lobster, and other crustaceans.",
            "A staple grain found in bread, pasta, and many baked goods.",
            "A common ingredient in processed foods and soy sauce.",
            "Sesame seeds and oil are common in breads, sauces, and snacks.",
            "Used in many condiments, dressings, and processed foods.",
            "A vegetable used in soups, stews, and spice blends.",
            "Preservatives used in dried fruits, wine, and some packaged foods.",
            "A legume found in some flours, bread, and gluten-free products."
        ]
    }
    allergies_df = pd.DataFrame(allergies_data)
    print("Allergy data created.")
except Exception as e:
    print(f"Error creating allergy DataFrame: {e}")
    allergies_df = pd.DataFrame()

# Define a Meal class to represent food items.
class Meal:
    """
    Represents a meal available in a shop.
    """
    def __init__(self, meal_id: str, meal_type: str, expiry_date: str, allergens: List[str], price: float):
        try:
            self.meal_id = meal_id
            self.meal_type = meal_type
            self.expiry_date = expiry_date
            self.allergens = allergens  # List of allergens present in the meal.
            self.price = price
            self.status = 'available'  # Initial status is "available".
        except Exception as e:
            print(f"Error initializing Meal object: {e}")

    # Check if the meal is still available and not expired.
    def is_available(self) -> bool:
        try:
            return self.status == 'available' and datetime.strptime(self.expiry_date, '%Y-%m-%d') > datetime.now()
        except Exception as e:
            print(f"Error checking meal availability: {e}")
            return False

# Define a User class to represent users in the system.
class User:
    """
    Represents a user in the food waste reduction app.
    """
    def __init__(self, user_id: str, name: str, email: str, location: Dict[str, float],
                 allergies: List[str] = [], balance: float = 100.0):
        try:
            self.user_id = user_id
            self.name = name
            self.email = email
            self.location = location  # User's latitude and longitude.
            self.allergies = allergies  # List of allergens to avoid.
            self.balance = balance  # User's account balance.
        except Exception as e:
            print(f"Error initializing User object: {e}")

    # Pay a shop for a meal if balance is sufficient.
    def pay_shop(self, shop: 'Shop', amount: float) -> bool:
        try:
            if self.balance >= amount:
                self.balance -= amount  # Deduct the amount.
                shop.receive_payment(amount)  # Add payment to shop's balance.
                return True
            return False  # Return false if insufficient funds.
        except Exception as e:
            print(f"Error processing payment: {e}")
            return False

    # Check if a meal is safe for the user based on allergens.
    def is_meal_safe(self, meal_allergens: List[str]) -> bool:
        try:
            return not any(allergy in meal_allergens for allergy in self.allergies)
        except Exception as e:
            print(f"Error checking meal safety: {e}")
            return False

    # Calculate distance to a shop using the Haversine formula.
    def calculate_distance(self, shop_location: Dict[str, float]) -> float:
        try:
            R = 6371  # Earth's radius in kilometers.
            lat1, lon1 = radians(self.location['latitude']), radians(self.location['longitude'])
            lat2, lon2 = radians(shop_location['latitude']), radians(shop_location['longitude'])
            dlat, dlon = lat2 - lat1, lon2 - lon1
            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            return R * c  # Distance in kilometers.
        except Exception as e:
            print(f"Error calculating distance: {e}")
            return float('inf')

# Define a Shop class to represent shops offering meals.
class Shop:
    """
    Represents a shop in the food waste reduction app.
    """
    def __init__(self, shop_id: str, name: str, location: Dict[str, float], balance: float = 0.0):
        try:
            self.shop_id = shop_id
            self.name = name
            self.location = location
            self.balance = balance
            self.meals = []  # List of meals available at the shop.
        except Exception as e:
            print(f"Error initializing Shop object: {e}")

    # Receive payment from a user.
    def receive_payment(self, amount: float) -> None:
        try:
            self.balance += amount
        except Exception as e:
            print(f"Error receiving payment: {e}")

    # Add a meal to the shop's inventory.
    def add_meal(self, meal: Meal) -> None:
        try:
            self.meals.append(meal)
        except Exception as e:
            print(f"Error adding meal: {e}")

    # Retrieve available meals that are not expired.
    def get_available_meals(self) -> List[Meal]:
        try:
            return [meal for meal in self.meals if meal.is_available()]
        except Exception as e:
            print(f"Error retrieving available meals: {e}")
            return []

# Merge sort for orders with error handling
def merge_sort(data: List['Order'], key: str) -> List['Order']:
    """
    Perform a merge sort on a list of orders based on a specified key.
    """
    try:
        if len(data) <= 1:
            return data
        mid = len(data) // 2
        left = merge_sort(data[:mid], key)
        right = merge_sort(data[mid:], key)
        return merge(left, right, key)
    except Exception as e:
        print(f"Error during merge sort: {e}")
        return []  # Return an empty list in case of failure

def merge(left: List['Order'], right: List['Order'], key: str) -> List['Order']:
    """
    Merge two sorted lists based on a specified key.
    """
    try:
        sorted_data = []
        while left and right:
            if getattr(left[0], key, None) is not None and getattr(right[0], key, None) is not None:
                if getattr(left[0], key) <= getattr(right[0], key):
                    sorted_data.append(left.pop(0))
                else:
                    sorted_data.append(right.pop(0))
            else:
                raise AttributeError(f"Key '{key}' not found in order objects.")
        sorted_data.extend(left)
        sorted_data.extend(right)
        return sorted_data
    except Exception as e:
        print(f"Error during merge: {e}")
        return left + right  # Return combined data if merge fails

# Order class with random timestamp
class Order:
    """
    Represents an order in the food waste reduction app.
    """
    def __init__(self, user: User, shop: Shop, meal: Meal):
        try:
            self.user_id = user.user_id
            self.user_name = user.name
            self.shop_id = shop.shop_id
            self.shop_name = shop.name
            self.meal_id = meal.meal_id
            self.meal_type = meal.meal_type
            self.price = meal.price
            now = datetime.now()
            future_time = now + timedelta(days=2)
            self.timestamp = (now + timedelta(
                seconds=random.randint(0, int((future_time - now).total_seconds()))
            )).strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(f"Error initializing Order object: {e}")
            self.timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Default to current time

# Linear search for nearest shop with a safe meal
def find_nearest_safe_shop(user: User, shops: List[Shop], preferred_meal_type: str) -> Tuple[Shop, Meal, float]:
    """
    Find the nearest shop with a safe meal for the user.
    """
    try:
        nearest_shop = None
        nearest_meal = None
        min_distance = float('inf')

        for shop in shops:
            distance = user.calculate_distance(shop.location)
            for meal in shop.get_available_meals():
                if meal.meal_type == preferred_meal_type and user.is_meal_safe(meal.allergens):
                    if distance < min_distance:
                        nearest_shop = shop
                        nearest_meal = meal
                        min_distance = distance
        return nearest_shop, nearest_meal, min_distance
    except Exception as e:
        print(f"Error during search for nearest safe shop: {e}")
        return None, None, float('inf')  # Return defaults in case of error

# Simulation function to generate users, shops, and orders
def simulate_users_and_shops(num_users: int, shop_data: List[Dict[str, str]]) -> Tuple[List[User], List[Shop]]:
    users = []
    for i in range(num_users):
        user_id = f"U{random.randint(10000, 99999)}"
        location = {
            "latitude": random.uniform(55.65, 55.72),  
            "longitude": random.uniform(12.48, 12.63)
        }
        allergies = random.sample(allergies_df['Allergen'].tolist(), k=random.randint(0, 3))
        balance = random.uniform(0, 7500)
        users.append(User(user_id, f"User_{user_id}", f"{user_id}@example.com", location, allergies, balance))

    shops = []
    for shop_info in shop_data:
        shop_id = f"S{random.randint(10000, 99999)}"
        location = {"latitude": shop_info['latitude'], "longitude": shop_info['longitude']}
        shops.append(Shop(shop_id, shop_info['name'], location))

    return users, shops

# Simulate users and shops
print("Simulating users and shops...")
try:
    num_users = 1000  # Adjust as needed
    users, shops = simulate_users_and_shops(num_users=num_users, shop_data=shop_data)
    print(f"Simulated {len(users)} users and {len(shops)} shops.")
except Exception as e:
    print(f"Error during user and shop simulation: {e}")

# Add meals to shops
print("Adding meals to shops...")
for shop in shops:
    for _ in range(random.randint(1, 5)):
        try:
            meal_id = f"M{random.randint(10000, 99999)}"
            meal_type = random.choice(['Sweet', 'Savory'])
            expiry_date = (datetime.now() + timedelta(days=random.randint(1, 7))).strftime('%Y-%m-%d')
            allergens = random.sample(allergies_df['Allergen'].tolist(), k=random.randint(0, 3))
            price = round(random.uniform(30, 150), 2)
            shop.add_meal(Meal(meal_id, meal_type, expiry_date, allergens, price))
        except Exception as e:
            print(f"Error adding meals to shop: {e}")

# Simulate orders
print("Simulating orders...")
orders = []
for user in users:
    try:
        preferred_meal_type = random.choice(['Sweet', 'Savory'])
        shop, meal, distance = find_nearest_safe_shop(user, shops, preferred_meal_type)
        if shop and meal and user.pay_shop(shop, meal.price):
            orders.append(Order(user, shop, meal))
    except Exception as e:
        print(f"Error processing user orders: {e}")

# Sort orders by timestamp
print("Sorting orders...")
sorted_orders = merge_sort(orders, key="timestamp")

# Save sorted orders to a CSV
print("Saving sorted orders to CSV...")
try:
    order_data = [{
        "user_id": order.user_id,
        "user_name": order.user_name,
        "shop_id": order.shop_id,
        "shop_name": order.shop_name,
        "meal_id": order.meal_id,
        "meal_type": order.meal_type,
        "price": order.price,
        "timestamp": order.timestamp
    } for order in sorted_orders if order.shop_name != "N/A"]

    order_df = pd.DataFrame(order_data)
    order_df.to_csv("sorted_orders.csv", index=False)
    print("Filtered sorted_orders saved to 'sorted_orders.csv'.")
except Exception as e:
    print(f"Error saving orders to CSV: {e}")